# Assignment #3

## 1. Data collection

crawler.py

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import os

def url_i(i):
    url = "https://www.findamasters.com/masters-degrees/msc-degrees/?PG={}".format(i)
    return url

def get_MasterURLS():
    for i in range(1, 401):
        links_list = set()
        url = url_i(i)
        response = requests.get(url)
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')
            # Find all the links in the HTML
            links = soup.find_all('a', href=True)
            # Print all the links
            for link in links:
                l = link['href']
                splitted = l.split("/")
                if splitted[-1][0:2] == "?i" and len(splitted[-1].split("#")) != 2:
                    links_list.add(l)
        if len(links_list) == 15:
            with open("links.txt", 'a') as f:
                for l in links_list:
                    f.write(l+ '\n')
        time.sleep(1)

def main():

    get_MasterURLS()

    j = 0

    with open('links.txt', 'r') as file:
        # Read the content of the file
        file_content = file.readlines()
        
    for page in range(1, 401):
        folder_name = "page_{}".format(page)
        if not os.path.exists(folder_name):
                os.makedirs(folder_name)
        for web in range(j, 15 + j):
            website = "https://www.findamasters.com" + file_content[web].replace("\n", "")
            response = requests.get(website)
            time.sleep(2)
            with open(folder_name + "/" + "page{}_master{}.html".format(page, web), 'a', encoding='utf-8') as file:
                file.write(response.text)
        j = page * 15
        print("Page " + str(page) + " has been completed.")
main()

parser.py

In [ ]:
from bs4 import BeautifulSoup
import re
import csv

def getAdministration(s):
    try:
        administration = s[-1]
    except IndexError:
        administration = " "
    return administration

def getNamefromRegExpr(atrr, script_tags):
    pattern = re.compile(r'DataLayerManager\.{}\s*=\s*"([^"]+)"'.format(atrr))
    val = ""
    for script_tag in script_tags:
        script_content = script_tag.get_text()
        match = pattern.search(script_content)
        if match:
            val = match.group(1)
    return val

def getDescription(soup):
    description_section = soup.find('div', class_='course-sections__description')
    if description_section:
        description_title = description_section.find('h2', class_='course-sections__title').text
        description_content = description_section.find('div', class_='course-sections__content').find('div', id='Snippet').text
        return description_content.replace("\n", "")
    else:
        return ""
    
def getFees(soup):
    fees_section = soup.find('div', class_='course-sections__fees')
    if fees_section:
        fees_title = fees_section.find('h2', class_='course-sections__title').text
        fees_content = fees_section.find('div', class_='course-sections__content').text
        return fees_content.replace("\n", "")
    else:
        return ""
    


def main():
    folder_names = ['page_' + str(i) for i in range(1, 401)]

    master = 0
    masters = []
    for folder in folder_names:
        for i in range(15):
            if master < 135:
                num_folder = folder[-1]
            elif master < 1485:
                num_folder = folder[-2] + folder[-1]
            else:
                num_folder = folder[-3] + folder[-2] + folder[-1]
            html_file = "page" + num_folder + "_master" + str(master)
            with open(folder+"/"+html_file + ".html", 'r', encoding='utf-8') as file:
                html_content = file.read()
            
            soup = BeautifulSoup(html_content, 'html.parser')

            script_tags = soup.find_all('script')
            courseName = soup.title.text.strip().split(' at ')[0]
            universityName = getNamefromRegExpr("dynamicInstitutionName", script_tags)
            facultyName = getNamefromRegExpr("dynamicDepartmentName", script_tags)
            isItFullTime = getNamefromRegExpr("dynamicStudyTypes", script_tags).split(",")[0:-1]
            description = getDescription(soup)
            startDate = getNamefromRegExpr("dynamicStudyTerms", script_tags)
            fees = getFees(soup)
            modality = getNamefromRegExpr("dynamicProgrammeTypes", script_tags)
            duration_elem = soup.find('span', class_='key-info__duration')
            if duration_elem:
                duration = duration_elem.text.strip()
            else:
                duration = ""
            city_elem = soup.find('a', class_='course-data__city')
            if city_elem:
                city = city_elem.text.strip()
            else:
                city = ""
            country = getNamefromRegExpr("dynamicLocationCountryName", script_tags)
            administration = getAdministration(getNamefromRegExpr("dynamicStudyTypes", script_tags).split(","))
            url_e = soup.find('link', rel='canonical')
            if url_e:
                url = url_e.get('href', 'URL not available')
            else:
                url = ""
            
            with open("courses_tsv/course_{}.tsv".format(master), 'a', encoding='utf-8-sig') as w:
                writer = csv.writer(w, delimiter = '\t')
                writer.writerow([courseName, universityName, facultyName, isItFullTime, description, startDate, fees, modality, duration, city, country, administration, url])
            master = master + 1
main()

## 2. Search engine 

### Libraries

In [1]:
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import pandas as pd
import os
import re
import requests
import numpy as np
import time
import folium
from folium import IFrame
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hazem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hazem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Script to create a DataFrame for the courses using the TSV files 

In [ ]:

# Path to the folder containing TSV files
folder_path = 'courses_tsv'

dfs = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.tsv'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read TSV file into a DataFrame
        df = pd.read_csv(file_path, sep='\t', header=None, names=[
            'courseName', 'universityName', 'facultyName', 'isItFullTime',
            'description', 'startDate', 'fees', 'modality', 'duration',
            'city', 'country', 'administration', 'url'
        ])
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

# Write the DataFrame to a CSV file
final_df.to_csv('masters_csv', index=False)


In [2]:
df = pd.read_csv('masters_csv')
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,"['Full Time', 'Part Time']",Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) International: £28,750 (To...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
1,"Agricultural, Environmental and Resource Econo...",University of Helsinki,International Masters Degree Programmes,['Full Time'],Goal of the pro­grammeAre you looking forward ...,September,Tuition fee per year (non-EU/EEA students): 15...,MSc,2 years,Helsinki,Finland,On Campus,https://www.findamasters.com/masters-degrees/c...
2,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,['Full Time'],"Our Accounting, Accountability & Financial Man...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
3,Energy and Environmental Sciences MSc,University of Groningen,Science and Engineering,['Full Time'],What are the causes of the various large scale...,September,Please see the university website for further ...,MSc,2 years full time,Groningen,Netherlands,On Campus,https://www.findamasters.com/masters-degrees/c...
4,Air Pollution Management and Control - MSc/PGDip,University of Birmingham,"School of Geography, Earth and Environmental S...","['Full Time', 'Part Time']",Our Air Pollution Management and Control MSc i...,September,Please see the university website for further ...,"PGDip,MSc","1 year full time, 2 years part time",Birmingham,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...


## 2. Search engine 

### Text Pre-processing

In [3]:
def preprocess_text(text):

    # Handling NaN/float values
    if not isinstance(text, str):
        return ''

    # Converting the text to lower case
    text = text.lower()

    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Breaking the text into words(tokens)
    words = nltk.word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Joining the list of words to form the processed text
    processed_text = ' '.join(words)

    return processed_text

### Processing the fees column

##### Getting the currencies' USD exchange rates using fastforext API


In [4]:
url = 'https://api.fastforex.io/fetch-all'
headers = {
    'accept': 'application/json',
}

# Include the API key as a query parameter
params = {
    'api_key': '56359a1bde-e4fd75ffb0-s4bxgk',
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    exchange_rates = response.json()
    

else:
    print('An error has occurred.')

exchange_rates = exchange_rates['results']

first_10_items = list(exchange_rates.items())[:10]

for key, value in first_10_items:
    print(f'{key}: {value}')


AED: 3.67288
AFN: 69.59229
ALL: 93.79502
AMD: 401.7486
ANG: 1.78657
AOA: 826.94932
ARS: 357.66399
AUD: 1.51772
AWG: 1.79938
AZN: 1.69103


##### Function that retrieves the tuition fees and its currency and converts it to USD 

In [5]:
# List of currency symbols and their corresponding currency codes
currency_symbols = {
    'USD': '$',
    'EUR': '€',
    'GBP': '£',
    'JPY': '¥',
    'AUD': 'A$',
    'CAD': 'C$',
    'CHF': 'CHF',
    'CNY': '¥',
    'SEK': 'kr',
    'NZD': 'NZ$',
    'INR': '₹',
    'SGD': 'S$',
    'HKD': 'HK$',
    'NOK': 'kr',
    'KRW': '₩',
    'TRY': '₺',
    'ZAR': 'R',
    'BRL': 'R$',
    'RUB': '₽',
    'MXN': 'Mex$',
    'ARS': '$',
    'IDR': 'Rp',
    'MYR': 'RM',
    'PHP': '₱',
    'THB': '฿',
    'CZK': 'Kč',
    'HUF': 'Ft',
    'PLN': 'zł',
    'ISK': 'kr',
    'DKK': 'kr',
    'AED': 'د.إ',
    'SAR': '﷼',
    'QAR': '﷼',
    'EGP': 'EGP',
    'ILS': '₪',
    'KWD': 'د.ك',
    'BHD': 'ب.د',
    'OMR': 'ر.ع.',
    'JOD': 'د.ا',
    'LBP': 'ل.ل',
    'JMD': 'J$',
    'TTD': 'TT$',
    'COP': 'COL$',
    'CLP': 'CLP$',
    'PEN': 'S/',
    'UYU': '$U',
    'CRC': '₡',
}


def extract_and_convert_fees(row, exchange_rates):

    fees_usd = 0  
    
# Regex to find the tuition fees

    for currency, symbol in currency_symbols.items():

        regex = rf'{symbol}(\d+(?:,\d{{3}})*(\.\d+)?)'
        matches = re.findall(regex, row)

        for match in matches:

            # Remove commas and convert to float
            num = float(match[0].replace(',', ''))
            # Multiply by the conversion rate to get the amount in USD
            if currency in exchange_rates:
                num *= exchange_rates[currency]
                fees_usd = max(fees_usd, num)

    return fees_usd




### Applying text-preprocessing to the column 'description' and adding the column 'fees_USD'

In [54]:
df_updated = df.copy()

# Preprocess the text in the description, courseName, universityName and city columns 
df_updated['description_lemmatized'] = df_updated['description'].apply(preprocess_text)
df_updated['courseName_lemmatized'] = df_updated['courseName'].apply(preprocess_text)
df_updated['universityName_lemmatized'] = df_updated['universityName'].apply(preprocess_text)
df_updated['city_lemmatized'] = df_updated['city'].apply(preprocess_text)

# Create the fees_USD column with the equivalent tuition in USD
df_updated['fees_USD'] = df_updated['fees'].apply(lambda row: extract_and_convert_fees(str(row), exchange_rates) if not pd.isnull(row) else np.nan)
df_updated['fees_USD'] = df_updated['fees_USD'].round(2)

In [55]:
# Masters degrees with tuition fees mentioned in the scraped data
dummy = df_updated[df_updated['fees_USD'] != 0]
dummy.head(8)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,description_lemmatized,courseName_lemmatized,universityName_lemmatized,city_lemmatized,fees_USD
0,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,"['Full Time', 'Part Time']",Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) International: £28,750 (To...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,7 million peopl estim die everi year due harm ...,air qualiti solut msc,univers leed,leed,22809.39
13,Analytical Chemistry (MSc),Sheffield Hallam University,Postgraduate Courses,"['Full Time', 'Part Time']",Apply for the part-time course here.Work in cu...,September,Our tuition fee for UK students starting full-...,"MSc,PGDip,PGCert","1 year full-time, 2 years part-time",Sheffield,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,appli parttim cours herework cuttingedg facil ...,analyt chemistri msc,sheffield hallam univers,sheffield,8179.64
14,Economics of Sustainability,Erasmus School of Economics,Masters Programmes,['Full Time'],Master in Economics of Sustainability: Focus o...,September,Tuition fees (2023-2024) €2.314 (Statutory fe...,MSc,1 year,Rotterdam,Netherlands,On Campus,https://www.findamasters.com/masters-degrees/c...,master econom sustain focu sustain develop eco...,econom sustain,erasmu school econom,rotterdam,14.72
18,Allergy & Clinical Immunology (Online) PGCert,University College Cork,College of Medicine and Health,['Part Time'],This new Postgraduate Certificate in Allergy a...,September,"The EU fee for this course is €3,130. The Non...","PGCert,PGDip,MSc",1 year part time,Cork,Ireland,Online,https://www.findamasters.com/masters-degrees/c...,new postgradu certif allergi clinic immunolog ...,allergi clinic immunolog onlin pgcert,univers colleg cork,cork,6352.37
19,Anaesthesia and Perioperative Science MSc,University College London,Division of Surgery and Interventional Science,['Part Time'],Register your interest in graduate study at UC...,"March, September","Part time - £7,050",MSc,2 years part time,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,regist interest graduat studi uclth msc anaest...,anaesthesia periop scienc msc,univers colleg london,london,5593.26
41,Analytics and Operations Research in Logistics...,Erasmus School of Economics,Masters Programmes,['Full Time'],Are you curious how logistics companies solve ...,September,Tuition fees (2023-2024) €2.314 (Statutory fe...,MSc,1 year,Rotterdam,Netherlands,On Campus,https://www.findamasters.com/masters-degrees/c...,curiou logist compani solv puzzl rout vehicl s...,analyt oper research logist msc,erasmu school econom,rotterdam,14.72
47,Energy Policy and Finance (MSc),University of St Andrews,Interdisciplinary Studies,"['Full Time', 'Part Time']",The MSc in Energy Ethics explores how we balan...,September,"Home £13,470 Overseas £27,230",MSc,"1 year full time, 2 year part time",St Andrews,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,msc energi ethic explor balanc energi demand c...,energi polici financ msc,univers st andrew,st andrew,21603.47
53,"Anthropology, Environment and Development MSc",University College London,Department of Anthropology,"['Full Time', 'Part Time']",Register your interest in graduate study at UC...,September,"Full time - £14,100 Part time - £7,050",MSc,"1 year full time, 2 years part time",London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,regist interest graduat studi uclth msc anthro...,anthropolog environ develop msc,univers colleg london,london,11186.52


### Conjunctive Querying

#### Creating the words vocabulary

In [89]:
# Create a vocabulary
vocabulary = set()

# Extract words from the 'description' column
for description in df_updated['description_lemmatized']:
     
     # Handling NaN/float values
     if isinstance(description, str):  
        words = description.split()
        vocabulary.update(words)

# Convert the vocabulary to a dictionary with words as keys and their index as values
vocabulary_dict = {idx: word for idx, word in enumerate(vocabulary)}


#### Creating the inverted index

In [71]:
# Create a dictionary to store word occurrences with row indices
inverted_index = {word: [] for word in vocabulary}

# Iterate over rows to find occurrences
i = 0
for idx, description in enumerate(df_updated['description_lemmatized']):

    # Handling NaN/float values
    if isinstance(description, str):  
        words = description.split()
        for word in words:
            if word in vocabulary:
                inverted_index[word].append(idx)
    

# Create a DataFrame from the dictionary
inverted_index = pd.DataFrame(list(inverted_index.items()), columns=['word', 'documents_IDs'])
inverted_index['documents_IDs'] = inverted_index['documents_IDs'].apply(lambda x: list(set(x)))
inverted_index.to_csv('inverted_index.csv', index=False)
inverted_index.head()


,word,documents_IDs
0,hurdl,[1533]
1,tradetheoret,[95]
2,transductionom,[3966]
3,techniquesstudi,[476]
4,master®,[4074]


#### Function to process the search query

In [11]:
def execute_query(query, inverted_index, df):
    
    # Apply text preprocessing to the query
    processed_query = preprocess_text(query)

    query_words = processed_query.split()

    # Find document_IDs for each word in the query using the inverted index
    result_document_ids = []
    
    for word in query_words:

        if word in inverted_index['word'].values:

            # Get the document_IDs for the current word
            word_document_ids = inverted_index.loc[inverted_index['word'] == word, 'documents_IDs'].values[0]

            # Add the document_IDs to the result list
            result_document_ids.extend(word_document_ids)

    # Remove duplicate document_IDs
    result_document_ids = list(set(result_document_ids))
    result_document_ids.sort()
    
    # Return the rows corresponding to the document_IDs
    return df_updated.loc[result_document_ids, ['courseName', 'universityName', 'description', 'city', 'url']]

In [12]:
sample_query = 'advanced knowledge'
result = execute_query(sample_query, inverted_index, df_updated)
print(f'Number of results: {len(result)}')
result.head()



Number of results: 2491


,courseName,universityName,description,city,url
2,"Accounting, Accountability & Financial Managem...",King’s College London,"Our Accounting, Accountability & Financial Man...",London,https://www.findamasters.com/masters-degrees/c...
7,Airline and Airport Corporate Management,London Metropolitan University,Why study this course?This innovative master's...,London,https://www.findamasters.com/masters-degrees/c...
11,Analogue and Digital Integrated Circuit Design...,Imperial College London,Our MSc in Analogue and Digital Integrated Cir...,London,https://www.findamasters.com/masters-degrees/c...
13,Analytical Chemistry (MSc),Sheffield Hallam University,Apply for the part-time course here.Work in cu...,Sheffield,https://www.findamasters.com/masters-degrees/c...
22,Allergy - MSc/PGDip/PGCert,Imperial College London,Allergy is an increasing global health problem...,London,https://www.findamasters.com/masters-degrees/c...


### Conjunctive querying and Ranking Score

#### TF-IDF

In [13]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(vocabulary=inverted_index['word'].values)
tfidf_matrix = tfidf_vectorizer.fit_transform(df_updated['description_lemmatized'])

# Convert the TF-IDF matrix to a DataFrame for better visualization
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=df_updated.index)
df_tfidf.head()

,hurdl,tradetheoret,transductionom,techniquesstudi,master®,companygain,careerwhen,offenc,worldwidey,kaust,...,40thi,bestqual,coorgan,beat,entrench,ul,ghana,materialsthi,datasetsthi,harm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.298859
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [124]:
# Creating the new inverted index
inverted_index_tfidf = inverted_index.copy()

# create a new column to store the document-tfidf pairs to match the format required
inverted_index_tfidf['documents_tfidf_pairs'] = inverted_index_tfidf.word.apply(lambda word: [(doc_id, df_tfidf.loc[doc_id, word]) for doc_id in df_tfidf.index if df_tfidf.loc[doc_id, word] > 0])
inverted_index_tfidf.head()

,word,documents_IDs,documents_tfidf_pairs
0,hurdl,[1533],"[(1533, 0.2029373291033616)]"
1,tradetheoret,[95],"[(95, 0.16391621097937675)]"
2,transductionom,[3966],"[(3966, 0.15916076607128118)]"
3,techniquesstudi,[476],"[(476, 0.21124066478102543)]"
4,master®,[4074],[]


##### Function that executes the query by applying cosine similarity 

In [18]:
def execute_query_modified(query, inverted_index_tfidf, df, df_tfidf, tfidf_vectorizer):

    # Preprocess the query
    query_words = preprocess_text(query).split()

    # Find related documents
    related_docs = set.intersection(*(
        set(id for id, _ in inverted_index_tfidf.loc[inverted_index_tfidf['word'] == word, 'documents_tfidf_pairs'].iloc[0])
        for word in query_words
        if word in inverted_index_tfidf['word'].values
    ))

    # Return an empty DataFrame if no related documents were found
    if not related_docs:
        return pd.DataFrame()

    # Calculate cosine similarity for each document
    query_vectorized = tfidf_vectorizer.transform([' '.join(query_words)])
    similarities = {id: cosine_similarity(query_vectorized, df_tfidf.loc[id].values.reshape(1, -1))[0][0] for id in related_docs}

    # Sort documents by their similarity score
    sorted_doc_ids = sorted(similarities, key=similarities.get, reverse=True)

    output = df.loc[sorted_doc_ids, ['courseName', 'universityName', 'description', 'city', 'fees_USD', 'url']]
    output['similarity'] = [similarities[id] for id in sorted_doc_ids]

    return output.reset_index(drop=True)


##### Executing the function using the query "advanced knowledge"

In [19]:
query = 'advanced knowledge'
query_result = execute_query_modified(query, inverted_index_tfidf, df_updated, df_tfidf, tfidf_vectorizer).head(15)
query_result



,courseName,universityName,description,city,fees_USD,url,similarity
0,Advanced Computing MSc,King’s College London,Our Advanced Computing MSc provides knowledge ...,London,0.00,https://www.findamasters.com/masters-degrees/c...,0.394639
1,Advancing Practice - MSc,University of Northampton,Our MSc Advancing Practice awards support the ...,Northampton,0.00,https://www.findamasters.com/masters-degrees/c...,0.380455
2,Advanced Healthcare Practice - MSc,Cardiff University,Why study this courseOur MSc Advanced Healthca...,Cardiff,0.00,https://www.findamasters.com/masters-degrees/c...,0.379754
3,Advanced Clinical Practice - MSc,Canterbury Christ Church University,Gain the knowledge and skills needed to become...,Canterbury,1289.08,https://www.findamasters.com/masters-degrees/c...,0.359401
4,Advanced Mechanical Engineering - MSc (Eng),University of Leeds,This course offers a broad range of advanced s...,Leeds,24820.77,https://www.findamasters.com/masters-degrees/c...,0.337821
5,Advanced Biomedical Engineering - MSc,University of Bradford,Biomedical engineering is a fast evolving inte...,Bradford,0.00,https://www.findamasters.com/masters-degrees/c...,0.316692
6,Advanced Clinical Practice - MSc,University of Northampton,Our MSc Advanced Clinical Practice course aims...,Northampton,0.00,https://www.findamasters.com/masters-degrees/c...,0.311848
7,Advanced Professional Practice (MSc),University of Gloucestershire,Our lecturers are research active experts who ...,Gloucester,0.00,https://www.findamasters.com/masters-degrees/c...,0.311319
8,Advanced Computing - MSc,University of the West of Scotland,Our MSc Advanced Computing course is designed ...,Glasgow,0.00,https://www.findamasters.com/masters-degrees/c...,0.305071
9,Advanced Clinical Practice (AHP) - MSc/PGDip/P...,Bangor University,The programme has been developed to enhance pr...,Bangor,0.00,https://www.findamasters.com/masters-degrees/c...,0.299051


## 3. Define a new score!

Adding a new Dataframe to our existing one to write a scoring function.

In [ ]:
qs_path = '2024 QS World University Rankings 1.1 (For qs.com).csv'
qs_df = pd.read_csv(qs_path)
qs_df.head()

Matching the collumn names on dataframes.

In [ ]:
if 'fees_USD' in dummy.columns:
    # Replace 'fees' column in df_updated with 'fees_USD' column from dummy
    df_updated['fees'] = dummy['fees_USD']

In [ ]:
df_updated = df_updated.merge(qs_df, left_on='universityName', right_on='Institution Name', how='left')

In [ ]:
# Dropping columns from df_updated
columns_to_drop = ['Academic Reputation Rank', 'Employer Reputation Rank','Faculty Student Rank','Faculty Student Rank', 'Citations per Faculty Rank', 'International Faculty Rank', 'International Students Rank', 'International Research Network Rank', 'Employment Outcomes Rank',  'Sustainability Rank', 'Overall SCORE']
df_updated = df_updated.drop(labels=columns_to_drop, axis=1)

In [ ]:
df_updated.head()

In [ ]:
# print all column names (factors) of the DataFrame
print(df_updated.columns.tolist())

So far we removed the unnecessery collumns and removed the Overal Score collumn (because we will define our own scoring function). Now lets start on scoring function.

In [ ]:
def calculate_score(document, query):
    score = 0

    # Weights for different components
    academic_weight = 0.15
    employer_weight = 0.15
    international_faculty_weight = 0.1
    international_students_weight = 0.1
    citations_weight = 0.1
    textual_relevance_weight = 0.4

    # Academic and Employer Reputation
    if not pd.isnull(document['Academic Reputation Score']):
        score += float(document['Academic Reputation Score']) / 100 * academic_weight
    if not pd.isnull(document['Employer Reputation Score']):
        score += float(document['Employer Reputation Score']) / 100 * employer_weight

    # International Faculty and Students Scores
    if not pd.isnull(document['International Faculty Score']):
        score += float(document['International Faculty Score']) / 100 * international_faculty_weight
    if not pd.isnull(document['International Students Score']):
        score += float(document['International Students Score']) / 100 * international_students_weight

    # Citations per Faculty
    if not pd.isnull(document['Citations per Faculty Score']):
        score += float(document['Citations per Faculty Score']) / 100 * citations_weight

    # Textual Relevance
    query_terms = set(query.lower().split())
    description_terms = set(document['description_lemmatized'].lower().split())
    common_terms = query_terms.intersection(description_terms)
    score += len(common_terms) / len(query_terms) * textual_relevance_weight

    # Normalization based on the total weights
    total_weight = academic_weight + employer_weight + international_faculty_weight + international_students_weight + citations_weight + textual_relevance_weight
    normalized_score = (score / total_weight) * 100  # Normalized to be out of 100

    return min(normalized_score, 100)  # Ensuring the score doesn't exceed 100

Re-Creating the inverted index

In [ ]:
# Create a dictionary to store word occurrences with row indices
result_dict = {word: [] for word in vocabulary}

# Iterate over rows to find occurrences
i = 0
for idx, description in enumerate(df_updated['description_lemmatized']):

    # Handling NaN/float values
    if isinstance(description, str):  
        words = description.split()
        for word in words:
            if word in vocabulary:
                result_dict[word].append(idx)
    
words_frequency = {}

for index, word in vocabulary_dict.items():
    if word in result_dict:
        words_frequency[index] = result_dict[word]

words_frequency

# Create a DataFrame from the dictionary
inverted_index = pd.DataFrame(list(result_dict.items()), columns=['word', 'documents_IDs'])
inverted_index['documents_IDs'] = inverted_index['documents_IDs'].apply(lambda x: list(set(x)))
inverted_index.to_csv('inverted_index.csv', index=False)
inverted_index.head()


Now we will add the similarity score calculation (using jaccard similarity) and sorting the master courses in our dataframe regarding on similarity scores and overall scores.

In [ ]:
# Define the preprocess_text function
def preprocess_text(text):
    
    # Handling NaN/float values
    if not isinstance(text, str):
        return ''

    # Converting the text to lower case
    text = text.lower()

    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Breaking the text into words(tokens)
    words = nltk.word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Joining the list of words to form the processed text
    processed_text = ' '.join(words)

    return processed_text

# Function to calculate Jaccard similarity score
def calculate_jaccard_similarity(query_terms, document_terms):
    intersection = query_terms.intersection(document_terms)
    union = query_terms.union(document_terms)
    if len(union) == 0:
        return 0
    return len(intersection) / len(union)

# Execute_query function
def execute_query(query, inverted_index, df):
    # Apply text preprocessing to the query
    processed_query = preprocess_text(query)
    query_words = processed_query.split()

    # Find document_IDs for each word in the query using the inverted index
    result_document_ids = []
    for word in query_words:
        if word in inverted_index['word'].values:
            # Retrieve and flatten the list of document IDs
            word_document_ids = inverted_index.loc[inverted_index['word'] == word, 'documents_IDs'].explode().tolist()
            result_document_ids.extend(word_document_ids)

    # Remove duplicate document_IDs and sort
    result_document_ids = sorted(set(result_document_ids))

    # Retrieve the rows corresponding to the document_IDs
    relevant_docs = df.loc[result_document_ids]

    # Process the descriptions and calculate similarity scores
    relevant_docs['processed_description'] = relevant_docs['description_lemmatized'].apply(preprocess_text)
    query_terms_set = set(processed_query.split())
    relevant_docs['similarity_score'] = relevant_docs['processed_description'].apply(
        lambda doc: calculate_jaccard_similarity(query_terms_set, set(doc.split()))
    )

    # Calculate relevance scores for each document
    relevant_docs['relevance_score'] = relevant_docs.apply(lambda doc: calculate_score(doc, query), axis=1)

    # Sort documents first by relevance score, then by similarity score
    sorted_results = relevant_docs.sort_values(by=['relevance_score', 'similarity_score'], ascending=[False, False])

    # Filter out additional courses from the same university, keeping only the most relevant one
    most_relevant_per_university = sorted_results.drop_duplicates(subset='universityName')

    # Select specific columns for the final output
    final_columns = ['courseName', 'universityName', 'description', 'url', 'fees_USD', 'city', 'similarity_score', 'relevance_score']
    final_results = most_relevant_per_university[final_columns]

    return final_results

In [ ]:
sample_query = 'advanced knowledge'
result = execute_query(sample_query, inverted_index, df_updated)
print(f'Number of results: {len(result)}')
result.head()

## 4. Master's Degrees visualization map

In [22]:
# function to get the coordinates and country of a list of cities
def get_city_country_with_coordinates(cities):
    # Dictionary to store cities and their coordinates and countries
    city_country_coordinates = {}

    for city in cities:
        # Geolocator instance
        geolocator = Nominatim(user_agent="my_geocoder")
        location = geolocator.geocode(city)

        if location:
            city_coordinates = (location.latitude, location.longitude)
            country = location.address.split(",")[-1].strip()
            city_country_coordinates[city] = (city_coordinates, country)

        else:
            # If the city is not found, set the coordinates to None
            city_country_coordinates[city] = (None, "")
        
        time.sleep(1)

    return city_country_coordinates

In [23]:
# list of all the cities
cities = set(df_updated['city'].values)
# dictionary of cities and their coordinates and countries
city_country_coordinates = get_city_country_with_coordinates(cities)

In [26]:
# Create a DataFrame
df_city_country_coordinates = pd.DataFrame(list(city_country_coordinates.items()), columns=['City', 'Data'])

# Split the 'Data' column into 'Coordinates' and 'Country'
df_city_country_coordinates[['Coordinates', 'Country']] = pd.DataFrame(df_city_country_coordinates['Data'].tolist(),
                                                                        index=df_city_country_coordinates.index)

# Drop the original 'Data' column
df_city_country_coordinates = df_city_country_coordinates.drop('Data', axis=1)

df_city_country_coordinates.head()

,City,Coordinates,Country
0,Groningen,"(53.2190652, 6.5680077)",Nederland
1,Kingston,"(17.9712148, -76.7928128)",Jamaica
2,Cambridge,"(52.2055314, 0.1186637)",United Kingdom
3,Heidelberg,"(49.4093582, 8.694724)",Deutschland
4,Brussels,"(50.8550018, 4.3512333761166175)",België / Belgique / Belgien


In [76]:
def create_map(df_city_country_coordinates, query_result):
    # Create a map centered around the first city in the coordinates dataframe
    map_center = df_city_country_coordinates['Coordinates'].iloc[0]
    my_map = folium.Map(location=map_center, zoom_start=2)

    # Add markers for each city in the query result dataframe
    for city_info in query_result['city'].unique():
        # Get the coordinates and country for the current city
        city_data = df_city_country_coordinates.loc[
            df_city_country_coordinates['City'] == city_info]
        city_coordinates = city_data['Coordinates'].values[0]
        city_country = city_data['Country'].values[0]

        # Get master courses information for the current city
        city_courses = query_result[query_result['city'] == city_info]

        # Create a label for the marker with master courses information
        label = f'<link rel="stylesheet" type="text/css" href="styles.css">'  # Link to external CSS
        label += f'<h4 style="text-align:center;">{city_info}, {city_country}</h4><br>'
        for index, course in city_courses.iterrows():
            label += f'<p><b>Course:</b> {course["courseName"]}</p>'
            label += f'<p><b>University:</b> {course["universityName"]}</p>'
            
            # Separate fees information
            fees_text = f'<p><b>Tuition in USD:</b> ${course["fees_USD"]:.2f}</p>' if course["fees_USD"] != 0.00 else \
                        f'<p><b>Tuition in USD:</b> Visit the website</p>'
            label += f'{fees_text}'

            # Separate URL information
            url_text = f'<p><b>URL:</b> <a href="{course["url"]}" target="_blank">{course["url"]}</a></p>'
            label += f'{url_text}<br><br>'

        # Create and add the marker to the map
        iframe = IFrame(html=label, width=500, height=500)
        popup = folium.Popup(iframe, max_width=500)
        folium.Marker(location=city_coordinates, popup=popup).add_to(my_map)

    # Display the map in the notebook
    display(my_map)


In [77]:
# Example usage
create_map(df_city_country_coordinates, query_result)

#### Map guide

- This tool provides students with an enhanced visualization of the geographical locations of the master's degree programs they are searching for.

- The map displays location icons on cities that host the master's degree programs of interest to the student.

- Upon clicking a location icon, a pop-up tab appears, presenting detailed information about the master's degree program. This includes the course name, university name, tuition fees in USD, and a clickable URL leading to the official website of the master's degree program.

## 5. Bonus: Complex search engine 

### Approach explanation

- In this question 3 more inverted indices will be made using the same procedures used in the first inverted index in question 2(Conjuctive querying only).

- The three inverted indices are for the columns 'CourseName', 'universityName', and 'city'.

- After applying the search for each column, the results are concatenated together in the same dataframe.

- Moreover, the searching filters(budget, countries, online modality) are applied to the aggregated dataframe.

- The duplicates are dropped and the search result is returned.

#### Vocabulary of course names

In [88]:
# Vocabulary for the 'courseName_lemmatized' column

# Create a vocabulary
course_vocab = set()

# Extract words from the 'courseName_lemmatized' column
for name in df_updated['courseName_lemmatized']:
     
     # Handling NaN/float values
     if isinstance(name, str):  
        words = name.split()
        course_vocab.update(words)

# Convert the vocabulary to a dictionary with words as keys and their index as values
course_vocab_dict = {idx: word for idx, word in enumerate(course_vocab)}


#### Inverted index for the course names vocabulary

In [84]:
# Create a dictionary to store word occurrences with row indices
inverted_index_course = {word: [] for word in course_vocab}

# Iterate over rows to find occurrences
i = 0
for idx, course in enumerate(df_updated['courseName_lemmatized']):

    # Handling NaN/float values
    if isinstance(course, str):  
        words = course.split()
        for word in words:
            if word in course_vocab:
                inverted_index_course[word].append(idx)
    
# Create a DataFrame from the dictionary
inverted_index_course = pd.DataFrame(list(inverted_index_course.items()), columns=['word', 'documents_IDs'])
inverted_index_course['documents_IDs'] = inverted_index_course['documents_IDs'].apply(lambda x: list(set(x)))
inverted_index_course.to_csv('inverted_index_course.csv', index=False)
inverted_index_course.head()


,word,documents_IDs
0,earth,"[3330, 2180, 2183, 2188, 2194, 4179, 2196, 219..."
1,tradit,[3132]
2,graphic,"[1346, 3677]"
3,grenobl,[1351]
4,mim,"[4280, 5538, 4759]"


#### Vocabulary of university names

In [87]:
# Vocabulary for the 'universityName_lemmatized' column

# Create a vocabulary
uni_vocab = set()

# Extract words from the 'universityName_lemmatized' column
for name in df_updated['universityName_lemmatized']:
     
     # Handling NaN/float values
     if isinstance(name, str):  
        words = name.split()
        uni_vocab.update(words)

# Convert the vocabulary to a dictionary with words as keys and their index as values
uni_vocab_dict = {idx: word for idx, word in enumerate(uni_vocab)}


#### Inverted index for the university names vocabulary

In [92]:
# Create a dictionary to store word occurrences with row indices
inverted_index_uni = {word: [] for word in uni_vocab}

# Iterate over rows to find occurrences
i = 0
for idx, uni in enumerate(df_updated['universityName_lemmatized']):

    # Handling NaN/float values
    if isinstance(uni, str):  
        words = uni.split()
        for word in words:
            if word in uni_vocab:
                inverted_index_uni[word].append(idx)
    
# Create a DataFrame from the dictionary
inverted_index_uni = pd.DataFrame(list(inverted_index_uni.items()), columns=['word', 'documents_IDs'])
inverted_index_uni['documents_IDs'] = inverted_index_uni['documents_IDs'].apply(lambda x: list(set(x)))
inverted_index_uni.to_csv('inverted_index_uni.csv', index=False)
inverted_index_uni.head()


,word,documents_IDs
0,meridional,[5449]
1,aristotl,[2808]
2,eiphi,"[5033, 5036, 5006, 4975, 4986, 4989, 4958, 5018]"
3,napier,"[5165, 4941]"
4,universidad,"[3074, 3717, 1823]"


#### Vocabulary of university cities

In [91]:
# Vocabulary for the 'city_lemmatized' column

# Create a vocabulary
cities_vocab = set()

# Extract words from the 'city_lemmatized' column
for city in df_updated['city_lemmatized']:
     
     # Handling NaN/float values
     if isinstance(city, str):  
        words = city.split()
        cities_vocab.update(words)

# Convert the vocabulary to a dictionary with words as keys and their index as values
cities_vocab_dict = {idx: word for idx, word in enumerate(cities_vocab)}


#### Inverted index for the university cities vocabulary

In [93]:
# Create a dictionary to store word occurrences with row indices
inverted_index_cities = {word: [] for word in cities_vocab}

# Iterate over rows to find occurrences
i = 0
for idx, city in enumerate(df_updated['city_lemmatized']):

    # Handling NaN/float values
    if isinstance(city, str):  
        words = city.split()
        for word in words:
            if word in cities_vocab:
                inverted_index_cities[word].append(idx)
    
# Create a DataFrame from the dictionary
inverted_index_cities = pd.DataFrame(list(inverted_index_cities.items()), columns=['word', 'documents_IDs'])
inverted_index_cities['documents_IDs'] = inverted_index_cities['documents_IDs'].apply(lambda x: list(set(x)))
inverted_index_cities.to_csv('inverted_index_cities.csv', index=False)
inverted_index_cities.head()


,word,documents_IDs
0,newport,"[5921, 1860, 4038, 5980, 3980, 5907, 5909, 597..."
1,lyon,"[2402, 2435]"
2,lausann,"[5507, 5509]"
3,nant,"[4258, 3108, 1810, 1266, 275, 5014, 4666, 4731..."
4,passau,"[4608, 4617, 4593, 4596, 4603, 4606]"


In [85]:
# A function that retrieves the user's search criteria

def get_search_criteria():

    # Get the coures name from the user
    course = input('For any criterion you dont want to specify please type "None", \n Enter the course name: ')

    # Get the university name from the user
    uni = input('Enter the university name: ')

    # Get the university city from the user
    city = input('Enter the university city: ')

    # Get the user's maximum budget
    budget = input('Enter your maximum budget (in USD): ')

    # Get the user's preferred duration
    started = input('Do want to consider only the courses that has started? (Yes/No): ')

    # Get the user's preferred modality
    modality = input('Enter your preferred modality (Online/On campus): ')

    # Get the user's preferred location
    location = input('Enter your preferred countries separated by a comma: ')
    
    # Split the string into a list using the comma as the delimiter
    location = ''.join(location)
    location = location.split(', ')
    

    # Create a dictionary to store the search criteria
    search_criteria = {
        'course': course,
        'uni': uni,
        'city': city,
        'budget': budget,
        'started': started,
        'modality': modality,
        'location': location,
    }

    return search_criteria

##### User Manual

 **Steps:**
 
    1. Course Name:
        - Prompt the user to enter the course name.

    2. University Name:
        - Prompt the user to enter the university name.

    3. University City:
        - Prompt the user to enter the university city.

    4. Budget
        - Prompt the user to enter their maximum budget in USD.

    5. Course Start
        - Ask the user if they want to consider only courses that have already started (Yes/No).

    6. Modality
        - Prompt the user to enter their preferred modality (Online/On campus).

    7. Location
        - Prompt the user to enter their preferred countries, separated by commas.

    - Input Validation
        - The user *MUST* type "None" for any criterion they don't want to specify.


In [25]:
# A dictionary containing the user's search criteria
filters_dict = get_search_criteria()

In [121]:
def execute_complex_query(query, inverted_index, df, filters_dict):
    
    # Apply text preprocessing to the query
    processed_query = preprocess_text(query)

    query_words = processed_query.split()

    # Find document_IDs for each word in the query using the inverted index
    result_document_ids = []
    
    for word in query_words:

        if word in inverted_index['word'].values:

            # Get the document_IDs for the current word
            word_document_ids = inverted_index.loc[inverted_index['word'] == word, 'documents_IDs'].values[0]

            # Add the document_IDs to the result list
            result_document_ids.extend(word_document_ids)

    # Remove duplicate document_IDs
    result_document_ids = list(set(result_document_ids))
    result_document_ids.sort()
    
    # Return the rows corresponding to the document_IDs
    result =  df.loc[result_document_ids, ['courseName', 'universityName', 'description', 'city', 'startDate' ,'country', 'administration', 
                                           'fees_USD',  'url']]

    # Applying the search criteria

    if filters_dict['course'] != 'None':
        course = preprocess_text(filters_dict['course'])
        course_words = course.split()

        # Find document_IDs for each word in the query using the inverted index
        result_document_ids = []

        for word in course_words:

            if word in inverted_index_course['word'].values:

                # Get the document_IDs for the current word
                word_document_ids = inverted_index_course.loc[inverted_index_course['word'] == word, 'documents_IDs'].values[0]

                # Add the document_IDs to the result list
                result_document_ids.extend(word_document_ids)

    # Remove duplicate document_IDs
    result_document_ids = list(set(result_document_ids))
    result_document_ids.sort()

    result1 =  df.loc[result_document_ids, ['courseName', 'universityName', 'description', 'city', 'startDate' ,'country', 'administration',
                                            'fees_USD', 'url']]

    if filters_dict['uni'] != 'None':
        uni = preprocess_text(filters_dict['uni'])
        uni_words = uni.split()

        # Find document_IDs for each word in the query using the inverted index
        result_document_ids = []

        for word in uni_words:

            if word in inverted_index_uni['word'].values:

                # Get the document_IDs for the current word
                word_document_ids = inverted_index_uni.loc[inverted_index_uni['word'] == word, 'documents_IDs'].values[0]

                # Add the document_IDs to the result list
                result_document_ids.extend(word_document_ids)

    # Remove duplicate document_IDs
    result_document_ids = list(set(result_document_ids))
    result_document_ids.sort()

    result2 =  df.loc[result_document_ids, ['courseName', 'universityName', 'description', 'city', 'startDate' ,'country', 'administration',
                                            'fees_USD',  'url']]
    


    if filters_dict['city'] != 'None':
        city = preprocess_text(filters_dict['city'])
        city_words = city.split()

        # Find document_IDs for each word in the query using the inverted index
        result_document_ids = []

        for word in city_words:

            if word in inverted_index_cities['word'].values:

                # Get the document_IDs for the current word
                word_document_ids = inverted_index_cities.loc[inverted_index_cities['word'] == word, 'documents_IDs'].values[0]

                # Add the document_IDs to the result list
                result_document_ids.extend(word_document_ids)

    # Remove duplicate document_IDs
    result_document_ids = list(set(result_document_ids))
    result_document_ids.sort()

    result3 =  df.loc[result_document_ids, ['courseName', 'universityName', 'description', 'city', 'startDate' ,'country', 'administration',
                                            'fees_USD',  'url']]

    # Aggregating the data from the different inverted indices
    result = pd.concat([result, result1, result2, result3], ignore_index=True)
    
    if result is not None:
        
    # Applying budget filter
        if filters_dict['budget'] != 'None':
            result = result.drop(result[result['fees_USD'] < float(filters_dict['budget'])].index, inplace=False)

        # applying modality filter
        if filters_dict['modality'] != 'None':
            result = result.drop(result[result['administration'] != filters_dict['modality']].index, inplace=False)

        # Applying university filter
        if filters_dict['uni'] != 'None':
                result = result.drop(result[result['universityName'] != filters_dict['uni']].index, inplace=False)

        # Applying countries filter 
        if filters_dict['location'] != 'None':
                result = result.drop(result[~result['country'].isin(filters_dict['location'])].index, inplace=False)

        # Applying city filter
        if filters_dict['city'] != 'None':
            result = result.drop(result[result['city'] != filters_dict['city']].index, inplace=False)

    #Drop duplicates    
    result = result.drop_duplicates()

    return result



In [122]:
# Example usage
filters_dict = { 'course': 'advanced knowledge', 'uni': 'None', 'city': 'London', 'budget': '10000', 'started': 'Yes', 'modality': 'Online',
                 'location': ['United Kingdom', 'England', 'Germany', 'France', 'Spain', 'Italy', 'Netherlands', 'Sweden', 'Switzerland', 'Belgium', 'Austria', 'Denmark',
                               'Norway', 'Finland', 'Ireland', 'Portugal', 'Greece', 'Poland', 'Romania', 'Czech Republic', 'Hungary', 'Bulgaria', 'Croatia', 'Slovakia']}

complex_search = execute_complex_query('advanced knowledge', inverted_index, df_updated, filters_dict)
complex_search.head(10)

,courseName,universityName,description,city,startDate,country,administration,fees_USD,url
459,Clinical Neurology (by Distance Learning) MSc,University College London,Register your interest in graduate study at UC...,London,September,United Kingdom,Online,11186.52,https://www.findamasters.com/masters-degrees/c...
3378,Business Information Systems Management MSc,Middlesex University,"Business information, and using it to increase...",London,"October, January",United Kingdom,Online,12455.91,https://www.findamasters.com/masters-degrees/c...
3421,International Public Policy MSc,Queen Mary University of London,Start date: January 2024The International Publ...,London,"May, September",United Kingdom,Online,10547.85,https://www.findamasters.com/masters-degrees/c...
3600,Developmental Transactional Analysis [DTA],Professional Development International,Transactional analysis is often associated wit...,London,Any Month,United Kingdom,Online,NaN,https://www.findamasters.com/masters-degrees/c...


## Command Line Question

As done in the previous assignment, we encourage using the command as a feature that Data Scientists must master.

Note: To answer the question in this section, you must strictly use command line tools. We will reject any other method of response. The final script must be placed in CommandLine.sh.

First, take the course_i.tsv files you created in point 1 and merge them using Linux commands (Hint: make sure that the first row containing the column names appears only once).

Now that you have your merged file named merged_courses.tsv, use Linux commands to answer the following questions:

Which country offers the most Master's Degrees? Which city?
How many colleges offer Part-Time education?
Print the percentage of courses in Engineering (the word "Engineer" is contained in the course's name).
Important note: You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable. Please run the script and include a screenshot of the output in the notebook for evaluation.

Script Purpose

The script is designed to merge data from multiple TSV (Tab-Separated Values) files into a single file and then perform specific analyses on the merged data. The analyses include finding which country and city offer the most Master's Degrees, counting how many colleges offer Part-Time education, and calculating the percentage of courses related to Engineering.

Script Breakdown

Setting Up the Environment

In [2]:
#!/bin/bash

This line indicates that the script should be run using the Bash shell, a common command-line interpreter.

DIR="/Users/onurozansunger/Desktop/ADM-HW3-Group-26/HW3/courses_tsv"
cd $DIR

These lines set the working directory to the folder containing the TSV files and then change to that directory.

Initializing the Merged File:

This command creates (or clears, if it already exists) a file named merged_courses.tsv. This file will be used to store the merged data.

Merging Data from Individual Files

This loop iterates over file names from course_0.tsv to course_5999.tsv.

For each file, it checks if the file exists (-f "$FILE").

If the file exists, it takes the first line from each file (head -n 1 "$FILE") and appends it to merged_courses.tsv. This is done because the data of interest is in the header line of each file.


Data Analysis

The script performs several analyses on the merged data:

1. Find the Country with the Most Master's Degrees:

Method: Utilizes awk to filter rows where the degree type column matches "MSc".

Process: Sorts and counts occurrences by country.

Output: Displays the country with the highest count.

2. Find the City with the Most Master's Degrees:

Method: Similar to the country analysis, but focuses on cities.

Process: Sorts and counts occurrences by city.

Output: Shows the city with the most Master's Degrees.

3. Count Colleges Offering Part-Time Education:

Method: Filters rows where the education mode column matches "Part Time".

Process: Counts unique college names that offer part-time education.

Output: Provides the total number of colleges offering part-time education.

4. Calculate the Percentage of Engineering Courses:

Method: Counts the total number of courses and the number of courses with "Engineering" or "Engineer" in their names.

Process: Calculates the percentage of engineering courses in relation to the total course count.

Output: Shows the percentage of engineering courses.

Output Results

The script concludes by echoing the results of the analyses to the console, providing insights into the dataset's contents in terms of degree types, education modes, and course subjects.

Full script:

Screenshot:

![foto.png](foto.png)


## Algorithmic Question

1. **Implement a code to solve the above mentioned problem.**

In [ ]:
sol_found = False

def reports_generation(list_hours, sumHours, partial_solution=[], idx=0, report=[]):
    global sol_found

    #Stop if a solution is found
    if sol_found:
        return

    if idx == len(list_hours):
        if sum(partial_solution) == sumHours:
            report.append(partial_solution.copy())
            sol_found = True
        return

    for element in list_hours[idx]:
        partial_solution.append(element)
        #Prune the partial solutions with an amount of hours > sumHours
        if sum(partial_solution) <= sumHours:
            reports_generation(list_hours, sumHours, partial_solution, idx + 1, report)
        partial_solution.pop()

    reports_generation(list_hours, sumHours, partial_solution, idx + 1, report)

    return report

def main():
    global sol_found

    line1 = input().split()
    d = int(line1[0])
    sumHours = int(line1[1])

    min_max_hours = []
    list_hours = []

    for i in range(d):
        line = input().split()
        min_max_hours.append((int(line[0]), int(line[1])))
        list_hours.append(list(range(min_max_hours[-1][0], min_max_hours[-1][1] + 1)))

    sol_found = False
    reports = reports_generation(list_hours, sumHours)

    if len(reports) == 0:
        print("NO")
    else:
        print("YES")
        print(" ".join(map(str, reports[0])))

main()

2. **What is the time complexity (the Big O notation) of your solution? Please provide a detailed explanation of how you calculated the time complexity.**


In this case, the complexity of the algorithm is exponential. Let's define L_i as the number of candidate hours to work for each day, for example, if we have a day with min_hour = 3 and max_hour = 5, the list will be [3, 4, 5] and therefore L will be equal to 3.

As we have D days, and we must generate all possible combinations between the D days, the cost consists of seeing if each element of the different lists would be part of the final solution or not, this is defined by the formula $2^{L_1 + L_2 + \ldots + L_N}$, so the cost is exponential.

Then the cost of generating the lists with the different available hours of each day is O(n), that is, linear with respect to the number of days.

Therefore, the complexity is O($2^{L_1 + L_2 + \ldots + L_N}$).

It must be remembered that this would be in the worst case, that is, finding a solution at the end such that the sum of the elements of the list found is equal to sumHours.

In our case, we make the code more efficient since when we find a first solution, we end the algorithm and also, those partial solutions that make the number of hours exceed sumHours are discarded and do not continue generating combinations.

3. **Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the two differ, which one is right? (why?)**


The answer of Bing AI is:

The time complexity of the given code is O($n^m$), where n is the maximum number of hours in a day and m is the number of days. This is because the code generates all possible combinations of hours for each day, which results in a time complexity that is exponential in the number of days. The reports_generation function is a recursive function that generates all possible combinations of hours for each day, and it is called for each day and for each possible number of hours in a day. Therefore, the time complexity is proportional to the number of possible combinations, which is n^m. However, please note that the actual time complexity may be less than this because the function returns as soon as a solution is found, and it also prunes partial solutions with a total number of hours greater than the required sum of hours. This pruning can significantly reduce the number of combinations that need to be generated and checked, especially if a solution is found early or if the required sum of hours is small compared to the total number of hours available. But in the worst-case scenario, the time complexity is still O($n^m$).

I think my solution is more concrete, the Bing AI solution, is more general, since it assumes that all lists have as many hours as the maximum hours of all days, however, my solution takes into account only the number of hours each day. That is, my solution represents the possible combinations of hours for all days while the AI solution represents the possible combinations of hours for each day.


4. **What do you think of the optimality of your code? Do you believe it is optimal? Can you improve? Please elaborate on your response.**


I have decided to use recursion to solve the problem. The code already includes some improvements, such as stopping generating solutions that already exceed sumHours or stopping when a solution has been found by using a boolean as a global variable. To make it more optimal, dynamic programming could have been used, using top down or bottom up, some function calls could have been saved in some container so as not to repeat calculations.